In [ ]:
####INSTALLATION DES MODULES ####

In [1]:
 !pip install Bio

In [2]:
pip install --upgrade ipywidgets jupyterlab_widgets

In [4]:
import importlib, Projet_Celia_TRANZEAT as projet

In [5]:
projet.run_tests()

test_valid_id (Projet_Celia_TRANZEAT.TestFetchUniprotRecord.test_valid_id) ... ok
test_field_types (Projet_Celia_TRANZEAT.TestGetProteinDictInfo.test_field_types)
Vérifie le type de chaque valeur du dictionnaire ... ok
test_keys_presence (Projet_Celia_TRANZEAT.TestGetProteinDictInfo.test_keys_presence)
Vérifie que toutes les clés attendues sont présentes dans le dictionnaire. ... ok
test_sequence_length (Projet_Celia_TRANZEAT.TestGetProteinDictInfo.test_sequence_length)
Vérifie que la longueur indiquée correspond bien à la longueur réelle de la séquence. ... ok
test_values_match_record (Projet_Celia_TRANZEAT.TestGetProteinDictInfo.test_values_match_record)
Vérifie que certaines valeurs du dictionnaire correspondent au record source. ... ok
test_keys_are_amino_acids (Projet_Celia_TRANZEAT.TestGetAAComposition.test_keys_are_amino_acids)
Vérifie que chaque clé est une lettre d’acide aminé. ... C:\Users\celia\anaconda3\Lib\site-packages\Bio\SeqUtils\ProtParam.py:106: BiopythonDeprecationWa

In [ ]:
### INTERFACE GRAPHIQUE: ######

In [6]:
###########################################################################################
#CITATION INTRODUCTIVE
###########################################################################################

import ipywidgets as widgets
from ipywidgets import Output, Layout, Button, Box
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML 

sortie = Output()


prenom = widgets.Text(
    value='',
    placeholder='Entrez votre prénom',
    description='Prénom:',
    disabled=False
)


def generer_citation(b):
    with sortie:
        sortie.clear_output()
        nom = prenom.value.strip() or "Ami·e"
        texte = f'{nom}, écris-moi l’identifiant UniProt qui t’intéresse et sélectionne ce que tu veux savoir dessus'
        display(HTML(f"""
        <div style="background-color:#FF69B4; padding:15px; border-radius:10px; text-align:center; margin-top:10px;">
            <h2 style="color:white; font-weight:600; margin:0;">{texte}</h2>
        </div>
        """))

###########################################################################################
#  Interface PRINCIPALE d’analyse de protéine
###########################################################################################

%load_ext autoreload
%autoreload 2
%matplotlib inline


id_input = widgets.Text(description="UniProt ID:", value="P69905")


CHOICES = ["Infos générales", "Composition AA", "Hydrophobicité", "Charge vs pH", "Motifs N-glyco"]
items_layout = Layout(width='auto')
box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid', width='25%')
FUCHSIA = "#FF1493"

selected_choice = widgets.Label(value=CHOICES[0])

def make_choice_button(label):
    btn = Button(description=label, layout=items_layout)
    btn.style.button_color = "#FFB6C1"   
    btn.style.font_weight = "bold"
    btn.style.font_size = "14px"
    btn.style.text_color = "#FF1493"   
    return btn

choice_buttons = [make_choice_button(lbl) for lbl in CHOICES]
choice_box = Box(children=choice_buttons, layout=box_layout)


line_color = widgets.ColorPicker(value="blue", description="Couleur trait:")
line_width = widgets.FloatSlider(value=1.5, min=0.1, max=5.0, step=0.1, description="Épaisseur:")
line_color.layout.display = "none"
line_width.layout.display = "none"


output = widgets.Output()


def on_choice_clicked(b):
    selected_choice.value = b.description
    show = selected_choice.value in ["Hydrophobicité", "Charge vs pH", "Composition AA"]
    line_color.layout.display = "" if show else "none"
    line_width.layout.display = "" if show else "none"

for btn in choice_buttons:
    btn.on_click(on_choice_clicked)

def on_load_clicked(b):
     with output:
        clear_output()
        protein_id = id_input.value.strip()
         
        if not protein_id:
            display(HTML("<div style='color:#FF1493; font-weight:bold;'>Erreur: Rentre un identifiant Uniprot</div>"))
            return

        try:
            record = projet.fetch_uniprot_record(protein_id)
        except Exception:
            display(HTML("<div style='color:#FF1493; font-weight:bold;'>Erreur: Rentre un identifiant Uniprot</div>"))
            return
            
        if not record:
            display(HTML("<div style='color:#FF1493; font-weight:bold;'>Erreur: Rentre un identifiant Uniprot</div>"))
            return
        
        choice = selected_choice.value

        if choice == "Infos générales":
            info = projet.get_protein_dictinfo(record)
            html = "<h3>Informations générales sur la protéine</h3><ul>"
            for key, value in info.items():
                html += f"<li><b>{key} :</b> {value}</li>"
            html += "</ul>"
            display(HTML(html))

        elif choice == "Composition AA":
            comp = projet.get_aa_composition(record)
            names = list(comp.keys())
            values = [v * 100 for v in comp.values()]
            plt.figure(figsize=(10,5))
            plt.bar(names, values, color=line_color.value, edgecolor="black", linewidth=line_width.value)
            plt.title("Composition en acides aminés")
            plt.xlabel("Acides aminés")
            plt.ylabel("Pourcentage (%)")
            plt.show()

        elif choice == "Hydrophobicité":
            hydro = projet.get_hydrophobicity(record)
            x = range(1, len(hydro)+1)
            plt.figure(figsize=(8,5))
            plt.plot(x, hydro, color=line_color.value, linewidth=line_width.value)
            plt.xlabel("Position des acides aminés")
            plt.ylabel("Score d'hydrophobicité")
            plt.title("Profil d'hydrophobicité")
            plt.grid(True)
            plt.show()

        elif choice == "Charge vs pH":
            pH, charge = projet.ph_charge(record)
            plt.figure(figsize=(8,5))
            plt.plot(pH, charge, color=line_color.value, linewidth=line_width.value)
            plt.xlabel("pH")
            plt.ylabel("Charge nette")
            plt.title("Charge nette en fonction du pH")
            plt.grid(True)
            plt.show()

        elif choice == "Motifs N-glyco":
            motifs = projet.find_nglyco_motif(record)
            if motifs:
                items = "".join(f"<li>Position <b>{p}</b></li>" for p in motifs)
                html = f"""
                <h3>Motifs N-glycosylation détectés</h3>
                <p><b>Total :</b> {len(motifs)}</p>
                <ul>{items}</ul>
                """
            else:
                html = """
                <h3>Motifs N-glycosylation détectés</h3>
                <p>Aucun motif <code>N-X-[S/T]</code> (avec X ≠ P) n’a été trouvé.</p>
                """
            display(HTML(html))

###########################################################################################
# Déclaration de tous les boutons  
###########################################################################################

# Bouton de citation
citation_button = widgets.Button(
    description="Générer la citation",
    style={'button_color': 'lightgreen', 'text_color': 'magenta', 'font_weight': 'bold', 'font_style': 'italic'},
    layout={'width': '200px', 'height': '50px', 'margin': '10px'}
)
citation_button.on_click(generer_citation)

# Bouton de chargement
load_button = widgets.Button(
    description="Charger",
    style={'button_color': 'lightgreen', 'text_color': 'magenta', 'font_weight': 'bold', 'font_style': 'italic'},
    layout={'width': '200px', 'height': '50px', 'margin': '10px'}
)
load_button.on_click(on_load_clicked)

###########################################################################################
# AFFICHAGE GLOBAL
###########################################################################################
display(HTML("""
<h1 style='text-align:center; color:#FF1493; font-weight:bold; margin-bottom:0;'>Projet d’analyse de protéine</h1>
<h3 style='text-align:center; color:#FFB6C1; font-weight:normal; margin-top:5px;'>Projet Python – GB4</h3>
<hr style='border:1px solid #FFB6C1; margin-top:10px; margin-bottom:10px;'>
<div style='position:fixed; bottom:10px; width:100%; text-align:center; color:#FFB6C1; font-size:14px;'>
</div>
"""))

display(prenom, citation_button, sortie)  
display(widgets.HBox([choice_box, widgets.VBox([widgets.Label("Choix sélectionné :"), selected_choice])]))
display(id_input, load_button, line_color, line_width, output)

Text(value='', description='Prénom:', placeholder='Entrez votre prénom')

Button(description='Générer la citation', layout=Layout(height='50px', margin='10px', width='200px'), style=Bu…

Output()

Text(value='P69905', description='UniProt ID:')

Button(description='Charger', layout=Layout(height='50px', margin='10px', width='200px'), style=ButtonStyle(bu…

ColorPicker(value='blue', description='Couleur trait:', layout=Layout(display='none'))

FloatSlider(value=1.5, description='Épaisseur:', layout=Layout(display='none'), max=5.0, min=0.1)

Output()

In [ ]:
#P29681
#Q0KHU2